<a href="https://colab.research.google.com/github/alex-smith-uwec/CS491/blob/main/Inauguration_TF_IDF_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF with Scikit-Learn
This notebook is adapted from one developed by [Melanie Walsh](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html)

 A popular and effective method for text analysis is called *term frequency–inverse document frequency*, often abbreviated *tf-idf*. Tf-idf is a method that tries to identify the most distinctively frequent or significant words in documents across a corpus.

In this lesson, we're going to learn how to calculate tf-idf scores using a collection of plain text (.txt) files and the Python library scikit-learn, which has a quick and nifty module called [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In this notebook we will calculate and normalize tf-idf scores for U.S. Inaugural Addresses with scikit-learn.

## Dataset

### U.S. Inaugural Addresses

<blockquote class="epigraph" style=" padding: 10px">

This is the meaning of our liberty and our creed; why men and women and children of every race and every faith can join in celebration across this magnificent Mall, and why a man whose father less than 60 years ago might not have been served at a local restaurant can now stand before you to take a most sacred oath.  So let us mark this day with remembrance of who we are and how far we have traveled.
<p class ="attribution">—Barack Obama, Inaugural Presidential Address, January 2009
    </p>
    
</blockquote>

During Barack Obama's Inaugural Address in January 2009, he mentioned "women" four different times, including in the passage quoted above. How distinctive is Obama's inclusion of women in this address compared to all other U.S. Presidents? This is one of the questions that we're going to try to answer with tf-idf.

## Breaking Down the TF-IDF Formula

But first, let's quickly discuss the tf-idf formula. The idea is pretty simple.

**tf-idf = term_frequency * inverse_document_frequency**

**term_frequency** = number of times a given term appears in document

**inverse_document_frequency** = log(total number of documents / number of documents with term) + 1**\***

You take the number of times a term occurs in a document (term frequency). Then you take the number of documents in which the same term occurs at least once divided by the total number of documents (document frequency), and you flip that fraction on its head (inverse document frequency). Then you multiply the two numbers together (term_frequency * inverse_document_frequency).

The reason we take the *inverse*, or flipped fraction, of document frequency is to boost the rarer words that occur in relatively few documents. Think about the inverse document frequency for the word "said" vs the word "pigeon." The term "said" appears in 13 (document frequency) of 14 (total documents) *Lost in the City* stories (14 / 13 --> a smaller inverse document frequency) while the term "pigeons" only occurs in 2 (document frequency) of the 14 stories (total documents) (14 / 2 --> a bigger inverse document frequency, a bigger tf-idf boost).

*There are a bunch of slightly different ways that you can calculate inverse document frequency. The version of idf that we're going to use is the [scikit-learn default](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer), which uses "smoothing" aka it adds a "1" to the numerator and denominator:

**inverse_document_frequency**  = log((1 + total_number_of_documents) / (number_of_documents_with_term +1)) + 1

<div class="margin sidebar" style=" padding: 10px">

> If smooth_idf=True (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(t) = log [ (1 + n) / (1 + df(t)) ] + 1.  
> -[scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

</div>

## TF-IDF with scikit-learn

[scikit-learn](https://scikit-learn.org/stable/index.html), imported as `sklearn`, is a popular Python library for machine learning approaches such as clustering, classification, and regression. Though we're not doing any machine learning in this lesson, we're nevertheless going to use scikit-learn's `TfidfVectorizer` and `CountVectorizer`.

Install scikit-learn (not necessary in Colab)

In [ ]:
# !pip install sklearn

Import necessary modules and libraries

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


#### Set Directory Path

Below we're setting the directory filepath that contains all the text files that we want to analyze.

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
my_path="/content/drive/MyDrive/CS491/Data/US_Inaugural_Addresses/"

In [174]:
file_names=!ls {my_path}
file_names=sorted(file_names)
print(file_names)

['01_washington_1789.txt\t       21_grant_1869.txt\t       41_truman_1949.txt', '02_washington_1793.txt\t       22_grant_1873.txt\t       42_eisenhower_1953.txt', '03_adams_john_1797.txt\t       23_hayes_1877.txt\t       43_eisenhower_1957.txt', '04_jefferson_1801.txt\t       24_garfield_1881.txt\t       44_kennedy_1961.txt', '05_jefferson_1805.txt\t       25_cleveland_1885.txt\t       45_johnson_1965.txt', '06_madison_1809.txt\t       26_harrison_1889.txt\t       46_nixon_1969.txt', '07_madison_1813.txt\t       27_cleveland_1893.txt\t       47_nixon_1973.txt', '08_monroe_1817.txt\t       28_mckinley_1897.txt\t       48_carter_1977.txt', '09_monroe_1821.txt\t       29_mckinley_1901.txt\t       49_reagan_1981.txt', '10_adams_john_quincy_1825.txt  30_roosevelt_theodore_1905.txt  50_reagan_1985.txt', '11_jackson_1829.txt\t       31_taft_1909.txt\t\t       51_bush_george_h_w_1989.txt', '12_jackson_1833.txt\t       32_wilson_1913.txt\t       52_clinton_1993.txt', '13_van_buren_1837.txt\t   

In [175]:
file_names

['01_washington_1789.txt\t       21_grant_1869.txt\t       41_truman_1949.txt',
 '02_washington_1793.txt\t       22_grant_1873.txt\t       42_eisenhower_1953.txt',
 '03_adams_john_1797.txt\t       23_hayes_1877.txt\t       43_eisenhower_1957.txt',
 '04_jefferson_1801.txt\t       24_garfield_1881.txt\t       44_kennedy_1961.txt',
 '05_jefferson_1805.txt\t       25_cleveland_1885.txt\t       45_johnson_1965.txt',
 '06_madison_1809.txt\t       26_harrison_1889.txt\t       46_nixon_1969.txt',
 '07_madison_1813.txt\t       27_cleveland_1893.txt\t       47_nixon_1973.txt',
 '08_monroe_1817.txt\t       28_mckinley_1897.txt\t       48_carter_1977.txt',
 '09_monroe_1821.txt\t       29_mckinley_1901.txt\t       49_reagan_1981.txt',
 '10_adams_john_quincy_1825.txt  30_roosevelt_theodore_1905.txt  50_reagan_1985.txt',
 '11_jackson_1829.txt\t       31_taft_1909.txt\t\t       51_bush_george_h_w_1989.txt',
 '12_jackson_1833.txt\t       32_wilson_1913.txt\t       52_clinton_1993.txt',
 '13_van_buren_1

In [176]:
# Split combined string by tabs and newlines, then filter out any empty strings
combined_string = ' '.join(file_names)
file_list = [name.strip() for name in combined_string.split() if name.strip() and '.' in name]

# Remove the extension from each file name
text_titles = sorted([name.split('.')[0] for name in file_list])



In [ ]:
text_titles

In [156]:
import glob
text_files=glob.glob(f"{my_path}/*.txt")
text_files=sorted(text_files)

## Calculate tf–idf

To calculate tf–idf scores for every word, we're going to use scikit-learn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

When you initialize TfidfVectorizer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf.

The recommended way to run `TfidfVectorizer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in text length, and overall produce more meaningful tf–idf scores. Smoothing and L2 normalization are actually the default settings for `TfidfVectorizer`, so to turn them on, you don't need to include any extra code at all.

Initialize TfidfVectorizer with desired parameters (default smoothing and normalization)

In [178]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')

Run TfidfVectorizer on our `text_files`

In [179]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)
#this is a sparse matrix

Make a DataFrame out of the resulting tf–idf vector, setting the "feature names" or words as columns and the titles as rows

In [180]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

Add column for document frequency aka number of times word appears in all documents

In [160]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

In [182]:
tfidf_slice = tfidf_df[['government', 'borders', 'people', 'war', 'honor','foreign', 'men', 'women', 'children','slave']]
tfidf_slice.sort_index().round(decimals=2)

,government,borders,people,war,honor,foreign,men,women,children,slave
01_washington_1789,0.11,0.00,0.05,0.00,0.00,0.00,0.02,0.00,0.00,0.00
02_washington_1793,0.06,0.00,0.05,0.00,0.08,0.00,0.00,0.00,0.00,0.00
03_adams_john_1797,0.16,0.00,0.19,0.01,0.10,0.12,0.04,0.00,0.00,0.00
04_jefferson_1801,0.16,0.00,0.01,0.01,0.04,0.00,0.04,0.00,0.00,0.00
05_jefferson_1805,0.03,0.00,0.00,0.04,0.00,0.06,0.01,0.00,0.02,0.00
06_madison_1809,0.00,0.00,0.02,0.02,0.05,0.05,0.00,0.00,0.00,0.00
07_madison_1813,0.04,0.00,0.04,0.25,0.02,0.02,0.00,0.00,0.00,0.00
08_monroe_1817,0.17,0.00,0.11,0.09,0.01,0.10,0.04,0.00,0.00,0.00
09_monroe_1821,0.08,0.00,0.06,0.11,0.02,0.04,0.01,0.00,0.01,0.02
10_adams_john_quincy_1825,0.15,0.00,0.06,0.05,0.01,0.08,0.03,0.00,0.00,0.00


Let's drop "OO_Document Frequency" since we were just using it for illustration purposes.

In [183]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

Let's reorganize the DataFrame so that the words are in rows rather than columns.

In [164]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,01_washington_1789,000,0.000000
1,01_washington_1789,03,0.000000
2,01_washington_1789,04,0.023259
3,01_washington_1789,05,0.000000
4,01_washington_1789,100,0.000000
...,...,...,...
521937,58_trump_2017,zachary,0.000000
521938,58_trump_2017,zeal,0.000000
521939,58_trump_2017,zealous,0.000000
521940,58_trump_2017,zealously,0.000000


In [184]:
tfidf_df = tfidf_df.stack().reset_index()

In [185]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

To find out the top 10 words with the highest tf–idf for every story, we're going to sort by document and tfidf score and then groupby document and take the first 10 values.

In [186]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,document,term,tfidf
3707,01_washington_1789,government,0.113681
4108,01_washington_1789,immutable,0.103883
4175,01_washington_1789,impressions,0.103883
6337,01_washington_1789,providential,0.103883
5631,01_washington_1789,ought,0.103728
...,...,...,...
518409,58_trump_2017,obama,0.120288
518766,58_trump_2017,people,0.112370
521001,58_trump_2017,thank,0.109171
513989,58_trump_2017,borders,0.107075


In [187]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

We can zoom in on particular words and particular documents.

In [190]:
top_tfidf[top_tfidf['term'].str.contains('women')]

,document,term,tfidf
503861,56_obama_2009,women,0.084859


It turns out that the term "women" is very distinctive in Obama's Inaugural Address.

In [ ]:
top_tfidf[top_tfidf['document'].str.contains('obama')]

,document,term,tfidf
425559,56_obama_2009,don,0.186313
424041,56_obama_2009,breeze,0.184416
428366,56_obama_2009,new,0.137266
426523,56_obama_2009,friends,0.136820
425563,56_obama_2009,door,0.133889
431878,56_obama_2009,word,0.131722
428268,56_obama_2009,mr,0.126821
426766,56_obama_2009,hand,0.125086
423971,56_obama_2009,blowing,0.110649
431030,56_obama_2009,things,0.110609


In [191]:
top_tfidf[top_tfidf['document'].str.contains('lincoln')]

,document,term,tfidf
163743,19_lincoln_1861,constitution,0.214478
170451,19_lincoln_1861,union,0.203738
163215,19_lincoln_1861,case,0.152422
169609,19_lincoln_1861,states,0.144861
167186,19_lincoln_1861,minority,0.131514
167805,19_lincoln_1861,people,0.130763
163377,19_lincoln_1861,clause,0.125738
165689,19_lincoln_1861,government,0.123837
169264,19_lincoln_1861,shall,0.123099
166740,19_lincoln_1861,law,0.122872


In [ ]:
top_tfidf[top_tfidf['document'].str.contains('kennedy')]

,document,term,tfidf
102696,44_kennedy_1961,government,0.186855
104812,44_kennedy_1961,people,0.162132
100750,44_kennedy_1961,constitution,0.158292
106616,44_kennedy_1961,states,0.135047
107458,44_kennedy_1961,union,0.132321
106799,44_kennedy_1961,suffrage,0.119992
104389,44_kennedy_1961,negro,0.118782
99777,44_kennedy_1961,authority,0.117232
100679,44_kennedy_1961,congress,0.112598
103747,44_kennedy_1961,law,0.103639


## Visualize TF-IDF

We can also visualize our TF-IDF results with the data visualization library Altair.

In [ ]:
# !pip install altair

Let's make a heatmap that shows the highest TF-IDF scoring words for each president, and let's put a red dot next to two terms of interest: "war" and "peace":

The code below was contributed by [Eric Monson](https://github.com/emonson). Thanks, Eric!

In [192]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)